In [4]:
import bs4
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from time import sleep

In [5]:
months=['October', 'November', 'December', 'January', 'February', 'March', 'April']
months_2=[10,11,12,1,2,3,4]
years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2018]
list_date=[]
for i in years:
    for k in months_2:
        if k in [10,11,12]:
            list_date.append(str([i,k]))
        if k in [1,2,3,4]:
            list_date.append(str([i+1,k]))
print(list_date)
seasons=['2005-06','2006-07','2007-08','2008-09','2009-10','2010-11','2011-12','2012-13',
         '2013-14','2014-15','2015-16','2016-17','2017-18','2018-19']

['[2005, 10]', '[2005, 11]', '[2005, 12]', '[2006, 1]', '[2006, 2]', '[2006, 3]', '[2006, 4]', '[2006, 10]', '[2006, 11]', '[2006, 12]', '[2007, 1]', '[2007, 2]', '[2007, 3]', '[2007, 4]', '[2007, 10]', '[2007, 11]', '[2007, 12]', '[2008, 1]', '[2008, 2]', '[2008, 3]', '[2008, 4]', '[2008, 10]', '[2008, 11]', '[2008, 12]', '[2009, 1]', '[2009, 2]', '[2009, 3]', '[2009, 4]', '[2009, 10]', '[2009, 11]', '[2009, 12]', '[2010, 1]', '[2010, 2]', '[2010, 3]', '[2010, 4]', '[2010, 10]', '[2010, 11]', '[2010, 12]', '[2011, 1]', '[2011, 2]', '[2011, 3]', '[2011, 4]', '[2011, 10]', '[2011, 11]', '[2011, 12]', '[2012, 1]', '[2012, 2]', '[2012, 3]', '[2012, 4]', '[2012, 10]', '[2012, 11]', '[2012, 12]', '[2013, 1]', '[2013, 2]', '[2013, 3]', '[2013, 4]', '[2013, 10]', '[2013, 11]', '[2013, 12]', '[2014, 1]', '[2014, 2]', '[2014, 3]', '[2014, 4]', '[2014, 10]', '[2014, 11]', '[2014, 12]', '[2015, 1]', '[2015, 2]', '[2015, 3]', '[2015, 4]', '[2015, 10]', '[2015, 11]', '[2015, 12]', '[2016, 1]', '[20

In [7]:
list_teams=['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'LA Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards']

In [8]:
def scrap_teams(season,month):
    url='https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season='+ season + '&SeasonType=Regular%20Season&Month=' + str(month)
    print(url)
    DRIVER_PATH = 'chromedriver'
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(url)
    driver.implicitly_wait(10)
    sleep(5)
    #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
    #print(driver.page_source)
    try:
        h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')
        #driver.close()
        #print("close")
        text = h3.text
        list_1=text.split('\n')
        #print(list_1)
        index = list_1.index('TEAM')
        del list_1[index:]
        header=list_1.pop(0).split(' ')
        #print(header)
        dataframe_list=[]
        #dataframe_list.append(header)
        i=0
        for k in range(len(list_1)):
            #print(k)
            if i==1:
                chiffres=list_1[k].split(' ')
                stat_equipe=[list_1[k-1]] + chiffres
                #print(stat_equipe)
                dataframe_list.append(stat_equipe)
                #print('dataframe', dataframe_list)
            i+=1
            if i==2:
                i=0
        #print(dataframe_list)
        df = pd.DataFrame(dataframe_list, columns = header)
        teams = df['TEAM'].tolist()
        if 'New Jersey Nets' in teams:
            teams = list(map(lambda x: x.replace('New Jersey Nets', 'Brooklyn Nets'), teams))
        if 'Charlotte Bobcats' in teams:
            teams = list(map(lambda x: x.replace('Charlotte Bobcats', 'Charlotte Hornets'), teams))
        if 'New Orleans/Oklahoma City Hornets' in teams:
            teams = list(map(lambda x: x.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans'), teams))
        if 'New Orleans Hornets' in teams:
            teams = list(map(lambda x: x.replace('New Orleans Hornets', 'New Orleans Pelicans'), teams))
        if 'Seattle SuperSonics' in teams:
            teams = list(map(lambda x: x.replace('Seattle SuperSonics', 'Oklahoma City Thunder'), teams))
        if 'Los Angeles Clippers' in teams:
            teams = list(map(lambda x: x.replace('Los Angeles Clippers', 'LA Clippers'), teams))           
        df['TEAM']=teams
        df=df.set_index('TEAM')
        df = df.sort_index()
        return df
    except:
        return None

In [47]:
date_n=0
df_final=pd.DataFrame(columns=list_teams)

for season in seasons:
#for season in ['2005-06']:
    for month in range(1,8):
        print(season + '  ---  ' + str(month))
        date=list_date[date_n]
        print(date)
        df_mois=scrap_teams(season,month)
        if isinstance(df_mois, pd.DataFrame):
            #print("est un dataframe")
            list_teams_2=df_mois.index.tolist()
            list_teams_2.sort()
            #print(list_teams_2)
            #print(list_teams)
            if list_teams_2 == list_teams:
                print("Mêmes équipes")
                values=[]
                for team in list_teams:
                    df2=df_mois.loc[[team]]
                    values.append(df2.to_dict('records')[0])
                    #print(values)
                df_final.loc[date]=values
            elif all(elem in list_teams for elem in list_teams_2) and list_teams_2 != list_teams:
                print("Toutes les équipes n'ont pas participé ce mois-ci")
                values=[]
                for team in list_teams:
                    if team in list_teams_2:
                        df2=df_mois.loc[[team]]
                        values.append(df2.to_dict('records')[0])                    
                    else:
                        values.append(np.nan)
                df_final.loc[date]=values
            else:
                print("---------------- ERREUR -----------------------")           
        else:
            print('Données pas dispos')
            df_final.loc[date]=[np.NAN]*30
        date_n+=1
print(df_final)

2005-06  ---  1
[2005, 10]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=1


<ipython-input-46-72aef711ecbe>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')


Données pas dispos
2005-06  ---  2
[2005, 11]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=2
Mêmes équipes
2005-06  ---  3
[2005, 12]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=3
Mêmes équipes
2005-06  ---  4
[2006, 1]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=4
Mêmes équipes
2005-06  ---  5
[2006, 2]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=5
Mêmes équipes
2005-06  ---  6
[2006, 3]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=6
Mêmes équipes
2005-06  ---  7
[2006, 4]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=7
Mêmes équipes
2006-07  ---  1
[2006, 10]
http

Mêmes équipes
2012-13  ---  4
[2013, 1]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=4
Mêmes équipes
2012-13  ---  5
[2013, 2]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=5
Mêmes équipes
2012-13  ---  6
[2013, 3]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=6
Mêmes équipes
2012-13  ---  7
[2013, 4]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=7
Mêmes équipes
2013-14  ---  1
[2013, 10]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2013-14&SeasonType=Regular%20Season&Month=1
Mêmes équipes
2013-14  ---  2
[2013, 11]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2013-14&SeasonType=Regular%20Season&Month=2
Mêmes équipes
2013-14  ---  3
[2013, 12]
https://w

In [49]:
df_final.to_csv('stats-equipes-par-mois.csv', sep=';')

In [ ]:
DRIVER_PATH = 'chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.nba.com/stats/teams/traditional/?sort=W_PCT&dir=-1&Season=2017-18&SeasonType=Regular%20Season&Month=1')

In [9]:
# ----------EXEMPLE
#print(scrap_teams('2018-19',5))
date_n=0
df_final=pd.DataFrame(columns=list_teams)

for season in ['2018-19']:
#for season in ['2005-06']:
    for month in [5]:
        print(season + '  ---  ' + str(month))
        date=list_date[date_n]
        print(date)
        df_mois=scrap_teams(season,month)
        print(df_mois)
        if isinstance(df_mois, pd.DataFrame):
            #print("est un dataframe")
            list_teams_2=df_mois.index.tolist()
            list_teams_2.sort()
            #print(list_teams_2)
            #print(list_teams)
            if list_teams_2 == list_teams:
                print("Mêmes équipes")
                values=[]
                for team in list_teams:
                    df2=df_mois.loc[[team]]
                    values.append(df2.to_dict('records')[0])
                    #print(values)
                df_final.loc[date]=values
            elif all(elem in list_teams for elem in list_teams_2) and list_teams_2 != list_teams:
                print("Toutes les équipes n'ont pas participé ce mois-ci")
                values=[]
                for team in list_teams:
                    if team in list_teams_2:
                        df2=df_mois.loc[[team]]
                        values.append(df2.to_dict('records')[0])                    
                    else:
                        values.append(np.nan)
                df_final.loc[date]=values
            else:
                print("---------------- ERREUR -----------------------")           
        else:
            print('Données pas dispos')
            df_final.loc[date]=[np.NAN]*30
        date_n+=1
df_final

2018-19  ---  5
[2005, 10]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2018-19&SeasonType=Regular%20Season&Month=5


C:\Users\nicol\AppData\Local\Temp/ipykernel_22392/4156471898.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')


                        GP   W  L  WIN%   MIN    PTS   FGM   FGA   FG%   3PM  \
TEAM                                                                           
Atlanta Hawks           12   5  7  .417  48.4  115.7  40.5  89.2  45.4  14.8   
Boston Celtics          11   5  6  .455  48.0  110.9  42.5  91.0  46.8  11.3   
Brooklyn Nets           10   4  6  .400  49.5  113.0  39.9  92.3  43.2  13.6   
Charlotte Hornets       11   4  7  .364  48.0  109.3  38.5  87.7  43.9  11.3   
Chicago Bulls           10   5  5  .500  48.0  116.0  43.1  88.2  48.9  10.3   
Cleveland Cavaliers     10   4  6  .400  49.5  109.4  39.6  91.3  43.4  12.9   
Dallas Mavericks        10   4  6  .400  48.0  105.9  37.1  85.1  43.6  14.2   
Denver Nuggets          11   7  4  .636  48.0  115.5  43.6  92.6  47.1  11.4   
Detroit Pistons         10   7  3  .700  48.0  113.6  40.5  86.8  46.7  14.8   
Golden State Warriors   11   7  4  .636  48.0  117.6  43.8  90.9  48.2  13.5   
Houston Rockets         12   8  4  .667 

,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,...,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
"[2005, 10]","{'GP': '12', 'W': '5', 'L': '7', 'WIN%': '.417...","{'GP': '11', 'W': '5', 'L': '6', 'WIN%': '.455...","{'GP': '10', 'W': '4', 'L': '6', 'WIN%': '.400...","{'GP': '11', 'W': '4', 'L': '7', 'WIN%': '.364...","{'GP': '10', 'W': '5', 'L': '5', 'WIN%': '.500...","{'GP': '10', 'W': '4', 'L': '6', 'WIN%': '.400...","{'GP': '10', 'W': '4', 'L': '6', 'WIN%': '.400...","{'GP': '11', 'W': '7', 'L': '4', 'WIN%': '.636...","{'GP': '10', 'W': '7', 'L': '3', 'WIN%': '.700...","{'GP': '11', 'W': '7', 'L': '4', 'WIN%': '.636...",...,"{'GP': '11', 'W': '6', 'L': '5', 'WIN%': '.545...","{'GP': '11', 'W': '8', 'L': '3', 'WIN%': '.727...","{'GP': '10', 'W': '6', 'L': '4', 'WIN%': '.600...","{'GP': '9', 'W': '1', 'L': '8', 'WIN%': '.111'...","{'GP': '9', 'W': '6', 'L': '3', 'WIN%': '.667'...","{'GP': '10', 'W': '5', 'L': '5', 'WIN%': '.500...","{'GP': '10', 'W': '3', 'L': '7', 'WIN%': '.300...","{'GP': '9', 'W': '8', 'L': '1', 'WIN%': '.889'...","{'GP': '9', 'W': '6', 'L': '3', 'WIN%': '.667'...","{'GP': '10', 'W': '3', 'L': '7', 'WIN%': '.300..."


In [11]:
df_mois=scrap_teams('2018-19',5)
df_mois

https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2018-19&SeasonType=Regular%20Season&Month=5


C:\Users\nicol\AppData\Local\Temp/ipykernel_22392/4156471898.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')


,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,3PM,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
TEAM,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,12,5,7,.417,48.4,115.7,40.5,89.2,45.4,14.8,...,34.8,45.4,24.6,15.8,6.8,4.8,5.9,22.8,22.0,-4.3
Boston Celtics,11,5,6,.455,48.0,110.9,42.5,91.0,46.8,11.3,...,34.3,43.1,26.5,10.9,8.2,5.5,3.3,19.7,18.7,-1.3
Brooklyn Nets,10,4,6,.400,49.5,113.0,39.9,92.3,43.2,13.6,...,36.1,45.3,26.5,14.9,6.0,4.9,5.9,20.3,22.4,-4.4
Charlotte Hornets,11,4,7,.364,48.0,109.3,38.5,87.7,43.9,11.3,...,33.0,43.7,20.7,11.2,7.9,5.6,4.6,18.3,19.9,-3.3
Chicago Bulls,10,5,5,.500,48.0,116.0,43.1,88.2,48.9,10.3,...,34.5,43.2,24.9,11.8,6.1,4.1,5.1,19.9,19.1,-0.1
Cleveland Cavaliers,10,4,6,.400,49.5,109.4,39.6,91.3,43.4,12.9,...,34.1,46.9,24.5,13.8,6.1,2.4,6.6,21.0,20.3,-4.2
Dallas Mavericks,10,4,6,.400,48.0,105.9,37.1,85.1,43.6,14.2,...,33.6,42.5,23.2,13.8,6.0,3.3,4.6,19.8,22.4,-4.8
Denver Nuggets,11,7,4,.636,48.0,115.5,43.6,92.6,47.1,11.4,...,34.9,47.5,29.9,14.0,7.5,4.5,4.6,21.4,21.5,3.1
Detroit Pistons,10,7,3,.700,48.0,113.6,40.5,86.8,46.7,14.8,...,34.0,44.0,23.1,12.5,6.5,4.4,4.8,21.3,21.2,6.5


In [12]:
values=[]
df_mois=pd.read_csv('test.csv')
df_mois=df_mois.set_index('TEAM')
df_mois
df2=df_mois.loc[['Utah Jazz']]
df2
values.append(df2.to_dict('records')[0])
values

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [13]:
#h1 = driver.find_element('/html/body/main/div/div/div[2]/div/div/nba-stat-table')
#h1 = driver.find_element(By.XPATH,'/html/body/main/div/div/div[2]/div/div/nba-stat-table')
#h1 = driver.find_element_by_name("stats-table-next")
#titles_element = driver.find_elements_by_xpath("//nba-stat-table")
#h2 = driver.find_element_by_class_name('someclass')
h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')
h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')

#h4 = driver.find_element_by_id('greatID')

NameError: name 'driver' is not defined

In [ ]:
text = h3.text

In [ ]:
list=text.split('\n')
print(list)

In [ ]:
index = list.index('TEAM')
del list[index:]
list

In [ ]:
header=list.pop(0).split(' ')
#print(list)
dataframe_list=[]
#dataframe_list.append(header)
i=0
for k in range(len(list)):
    print(k)
    if i==2:
        chiffres=list[k].split(' ')
        stat_equipe=[list[k-1]] + chiffres
        print(stat_equipe)
        dataframe_list.append(stat_equipe)
        print('dataframe', dataframe_list)
        
    i+=1
    if i==3:
        i=0
print(dataframe_list)  

In [ ]:
df = pd.DataFrame(dataframe_list, columns = header)
df=df.set_index('TEAM')
print(df)
# print dataframe.

In [ ]:
df.to_dict('records')

In [ ]:
df.to_csv("test.csv")

In [ ]:
df2=df.loc[['Boston Celtics']]
a=df2.to_dict('records')[0]

In [ ]:
print(a)